<a href="https://colab.research.google.com/github/kifoo/Sieci_neuronowe/blob/main/SN_projekt1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Kod implementacji płystkiej sieci neuronowej

In [ ]:
!pip install ucimlrepo
!pip install tensorflow

Importing required libraries

In [ ]:
import numpy as np
import pandas as pd
import os
import csv
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow import keras
from keras import models, layers, callbacks
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from ucimlrepo import fetch_ucirepo

Function fetching dataset from https://archive.ics.uci.edu/dataset/602/dry+bean+dataset

In [ ]:
def data_fetch():
    # fetch dataset
    dry_bean_dataset = fetch_ucirepo(id=602)

    # data (as pandas dataframes)
    X = dry_bean_dataset.data.features
    Y = dry_bean_dataset.data.targets

    label_encoder = LabelEncoder()
    Y = label_encoder.fit_transform(Y)
    return X, Y

Function defining the neural network model

In [ ]:
# Define the neural network model
def create_model(input_shape, output_shape, nuerons_output, hidden_layers):
    model = Sequential()
    model.add(Dense(nuerons_output, activation='relu', input_shape=(input_shape,)))
    for i in range(hidden_layers):
        model.add(Dense(nuerons_output, activation='relu'))
    model.add(Dense(output_shape, activation='softmax'))
    return model

Creating the neural network model, training it and testing it accuracy

In [ ]:
def neural_network(opt_function, loss_function, nuerons_output, hidden_layers, epochs):
    # This function return the dataframe
    X, Y = data_fetch()

    # Split the dataset into a training set and a testing set
    X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5)

    # Normalize the inputs
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # print("========== Train ==========")
    # print(y_train.shape[0])
    # print("========== Test ==========")
    # print(X_train.shape[1])

    # Create the neural network model
    model = create_model(input_shape=X_train.shape[1], output_shape=7, nuerons_output=nuerons_output, hidden_layers=hidden_layers)

    # Compile the model
    model.compile(optimizer=opt_function, loss=loss_function, metrics='accuracy')

    # Train the model
    history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs)

    # Test the model
    _, accuracy = model.evaluate(X_test, y_test)
    print(f"Test Accuracy: {accuracy}")
    return accuracy

Loss function exapmles:

**'mean_squared_error' or 'mse':** Measures the average squared difference between
the true and predicted labels. It is useful for regression problems.

**'binary_crossentropy':** Applies a binary cross-entropy between the true and predicted labels. It is useful for binary classification problems.

**'categorical_crossentropy':** Applies a categorical cross-entropy between the true and predicted labels. It is useful for multi-class classification problems.

**'hinge':** Defines a hinge loss function that measures the difference between the predicted labels and true labels.

**'logcosh':** Defines a logarithm of the hyperbolic cosine loss function.

**'sparse_categorical_crossentropy':** Similar to 'categorical_crossentropy', but it can accept labels with integer values instead of one-hot encoded vectors.

**'kl_divergence':** Calculates the Kullback-Leibler divergence, which quantifies the difference between two probability distributions.

**'poisson':** Calculates the Poisson loss between the true and predicted labels.

In [ ]:
def write_data(neuron_outputs,hidden_layers,epochs,acc):
    csv_file_path ="content/data.csv"

    with open(csv_file_path, 'a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([neuron_outputs, hidden_layers, epochs, acc])
        print("ok\n")


def main():
    loss = ['mean_squared_error', 'binary_crossentropy', 'categorical_crossentropy', 'hinge', 'logcosh', 'sparse_categorical_crossentropy', 'kl_divergence']
    neuron_outputs = [2, 4, 8, 12, 16]
    hidden_layers = [0, 1, 2]  # góra-cy, ale liczymy od 0 jak prawdziwi informatycy XDD
    epochs = [5, 10, 25, 50]

    # Ej bo tu wychodzi jakieś 60 danych XDD Dokładnie to 63, nie wiem czemu XDD
    min_acc = 1
    min_data = np.array([[0], [0], [0], [0.0]])
    max_acc = 0
    max_data = np.array([[0], [0], [0], [0.0]])
    write_data("Neuron Output","Hidden Layers","Epochs","Accuracy")

    for a in range(len(neuron_outputs)):
          for b in range(len(hidden_layers)):
              for c in range(len(epochs)):
                  acc = neural_network(opt_function='Adam', loss_function=loss[5], nuerons_output=neuron_outputs[a], hidden_layers=hidden_layers[b], epochs=epochs[c])
                  if( acc > max_acc):
                      max_acc = acc
                      max_data = [[neuron_outputs[a]], [hidden_layers[b]], [epochs[c]], [acc]]
                  if( acc < min_acc):
                      min_acc = acc
                      min_data = [[neuron_outputs[a]], [hidden_layers[b]], [epochs[c]], [acc]]
                  write_data(neuron_outputs[a],hidden_layers[b],epochs[c],acc)

    print(f"Min accuracy: {min_data} \nMax accuracy: {max_data}")

main()

ok

Epoch 1/5


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


213/213 [==============================] - 4s 11ms/step - loss: 1.8853 - accuracy: 0.3325 - val_loss: 1.7340 - val_accuracy: 0.3819
Epoch 2/5
213/213 [==============================] - 2s 9ms/step - loss: 1.6535 - accuracy: 0.3919 - val_loss: 1.5581 - val_accuracy: 0.4030
Epoch 3/5
213/213 [==============================] - 1s 5ms/step - loss: 1.5245 - accuracy: 0.4314 - val_loss: 1.4518 - val_accuracy: 0.4486
Epoch 4/5
213/213 [==============================] - 1s 6ms/step - loss: 1.4249 - accuracy: 0.4977 - val_loss: 1.3573 - val_accuracy: 0.5195
Epoch 5/5
213/213 [==============================] - 0s 2ms/step - loss: 1.2698 - accuracy: 0.5639
Test Accuracy: 0.5639141798019409
ok

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


213/213 [==============================] - 2s 4ms/step - loss: 1.8547 - accuracy: 0.4215 - val_loss: 1.7579 - val_accuracy: 0.4614
Epoch 2/10
213/213 [==============================] - 1s 4ms/step - loss: 1.6100 - accuracy: 0.4891 - val_loss: 1.4598 - val_accuracy: 0.5538
Epoch 3/10
213/213 [==============================] - 1s 4ms/step - loss: 1.3486 - accuracy: 0.5691 - val_loss: 1.2546 - val_accuracy: 0.6183
Epoch 4/10
213/213 [==============================] - 1s 4ms/step - loss: 1.1872 - accuracy: 0.6400 - val_loss: 1.1207 - val_accuracy: 0.6759
Epoch 5/10
213/213 [==============================] - 1s 4ms/step - loss: 1.0736 - accuracy: 0.6901 - val_loss: 1.0176 - val_accuracy: 0.7148
Epoch 6/10
213/213 [==============================] - 1s 4ms/step - loss: 0.9799 - accuracy: 0.7212 - val_loss: 0.9329 - val_accuracy: 0.7336
Epoch 7/10
213/213 [==============================] - 1s 5ms/step - loss: 0.9048 - accuracy: 0.7375 - val_loss: 0.8646 - val_accuracy: 0.7530
Epoch 8/10
213/21

...

213/213 [==============================] - 3s 7ms/step - loss: 1.1954 - accuracy: 0.5834 - val_loss: 0.6331 - val_accuracy: 0.7831
Epoch 2/10
213/213 [==============================] - 1s 4ms/step - loss: 0.4376 - accuracy: 0.8676 - val_loss: 0.3131 - val_accuracy: 0.9013
Epoch 3/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2796 - accuracy: 0.9083 - val_loss: 0.2404 - val_accuracy: 0.9195
Epoch 4/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2355 - accuracy: 0.9217 - val_loss: 0.2284 - val_accuracy: 0.9208
Epoch 5/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2210 - accuracy: 0.9221 - val_loss: 0.2124 - val_accuracy: 0.9255
Epoch 6/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2125 - accuracy: 0.9246 - val_loss: 0.2180 - val_accuracy: 0.9240
Epoch 7/10
213/213 [==============================] - 1s 4ms/step - loss: 0.2090 - accuracy: 0.9246 - val_loss: 0.2038 - val_accuracy: 0.9259
Epoch 8/10
213/21

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Epoch 1/25
213/213 [==============================] - 3s 8ms/step - loss: 1.1328 - accuracy: 0.5919 - val_loss: 0.5896 - val_accuracy: 0.7618
Epoch 2/25
213/213 [==============================] - 1s 4ms/step - loss: 0.4593 - accuracy: 0.8337 - val_loss: 0.3753 - val_accuracy: 0.8858
Epoch 3/25
213/213 [==============================] - 1s 4ms/step - loss: 0.3222 - accuracy: 0.8938 - val_loss: 0.2893 - val_accuracy: 0.9099
Epoch 4/25
213/213 [==============================] - 1s 4ms/step - loss: 0.2617 - accuracy: 0.9104 - val_loss: 0.2548 - val_accuracy: 0.9138
Epoch 5/25
213/213 [==============================] - 1s 4ms/step - loss: 0.2359 - accuracy: 0.9143 - val_loss: 0.2332 - val_accuracy: 0.9189
Epoch 6/25
213/213 [==============================] - 1s 4ms/step - loss: 0.2236 - accuracy: 0.9168 - val_loss: 0.2250 - val_accuracy: 0.9198
Epoch 7/25
213/213 [==============================] - 1s 4ms/step - loss: 0.2164 - accuracy: 0.9183 - val_loss: 0.2175 - val_accuracy: 0.9257
Epoch 

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:116: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


213/213 [==============================] - 2s 4ms/step - loss: 1.2644 - accuracy: 0.5649 - val_loss: 0.6313 - val_accuracy: 0.8234
Epoch 2/50
213/213 [==============================] - 1s 4ms/step - loss: 0.4186 - accuracy: 0.8675 - val_loss: 0.3212 - val_accuracy: 0.8888
Epoch 3/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2703 - accuracy: 0.9073 - val_loss: 0.2638 - val_accuracy: 0.9095
Epoch 4/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2315 - accuracy: 0.9190 - val_loss: 0.2427 - val_accuracy: 0.9165
Epoch 5/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2148 - accuracy: 0.9227 - val_loss: 0.2353 - val_accuracy: 0.9185
Epoch 6/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2086 - accuracy: 0.9229 - val_loss: 0.2368 - val_accuracy: 0.9149
Epoch 7/50
213/213 [==============================] - 1s 4ms/step - loss: 0.2040 - accuracy: 0.9239 - val_loss: 0.2283 - val_accuracy: 0.9217
Epoch 8/50
213/21

Min accuracy: [[2], [2], [50], [0.2634440064430237]]

Max accuracy: [[16], [1], [50], [0.9315310120582581]]